In [1]:
import sqlalchemy as sa
import pandas as pd

database_management_sys = sa.create_engine('sqlite:///../../../../Desktop/rerun-data-no-long-run/se/from_csv.db')
connect_dbms_to_db = database_management_sys.connect()

In [2]:
data = pd.read_sql("folk_theorem_experiment", connect_dbms_to_db)
len(data)

513400

In [7]:
data.tail()

,experiment_number,number_of_players,tournament_player_set,player_strategy_name,is_long_run_time,is_stochastic,memory_depth_of_strategy,prob_of_game_ending,payoff_matrix,num_of_repetitions,num_of_equilibria,nash_equilibria,least_prob_of_defection,greatest_prob_of_defection,noise,warning_message
513395,131399,6,119,"Grumpy: Nice, 10, -10",0,0,Inf,0.999,2.4405 2.504 2.508 2.394 1.768 1.722...,500,1,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.,1.0,1.0,0.4,None
513396,131399,6,119,Retaliate 2: 0.08,0,0,Inf,0.999,2.4405 2.504 2.508 2.394 1.768 1.722...,500,1,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.,1.0,1.0,0.4,None
513397,131399,6,119,Cycle Hunter,0,0,Inf,0.999,2.4405 2.504 2.508 2.394 1.768 1.722...,500,1,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.,1.0,1.0,0.4,None
513398,131399,6,119,Cycler DC,0,0,1,0.999,2.4405 2.504 2.508 2.394 1.768 1.722...,500,1,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.,1.0,1.0,0.4,None
513399,131399,6,119,Defector,0,0,0,0.999,2.4405 2.504 2.508 2.394 1.768 1.722...,500,1,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.,1.0,1.0,0.4,None


In [3]:
table_headings = pd.read_sql(
    "folk_theorem_experiment", connect_dbms_to_db
).columns.tolist()
table_headings

['experiment_number',
 'number_of_players',
 'tournament_player_set',
 'player_strategy_name',
 'is_long_run_time',
 'is_stochastic',
 'memory_depth_of_strategy',
 'prob_of_game_ending',
 'payoff_matrix',
 'num_of_repetitions',
 'num_of_equilibria',
 'nash_equilibria',
 'least_prob_of_defection',
 'greatest_prob_of_defection',
 'noise',
 'warning_message']

In [10]:
max_tournament_player_set = """
    SELECT MAX(tournament_player_set) FROM folk_theorem_experiment
"""
max_num_of_player_sets = pd.read_sql(max_tournament_player_set, connect_dbms_to_db)
max_num_of_player_sets

,MAX(tournament_player_set)
0,99


In [ ]:
max_experiment_num = """
    SELECT MAX(experiment_number) FROM folk_theorem_experiment
"""
max_num_of_experiments = pd.read_sql(max_experiment_num, connect_dbms_to_db)
max_num_of_experiments